### 알림 신호를 위한 랙의 좌표값 설정하기
- GUI에서 좌표값을 받아야함!

In [6]:
# 일단 고정되어있다고 가정하자
# cam 5
# u_x1, u_y1, u_x2, u_y2 = 137, 32, 437, 189
# d_x1, d_y1, d_x2, d_y2 = 110, 253, 363, 515

# cam 6
u_x1, u_y1, u_x2, u_y2 = 866, 294, 1111, 33
l_x1, l_y1, l_x2, l_y2 = 897, 293, 1232, 547

In [7]:
import cv2, time
import numpy as np
from ultralytics import YOLO

In [17]:
# 저장된 모델 불러오기
num = 7
model = YOLO(f'./runs/detect/train{num}/weights/best.onnx')

# 비디오 파일 로드
file = "C:/Users/kdp/Desktop/test_video_01.mp4"
video = cv2.VideoCapture(file)

# 프레임 간격 설정 (예: 매 10번째 프레임 읽기)
frame_interval = 1000

# 새로운 비디오 파일 초기화
fourcc = cv2.VideoWriter_fourcc(*'XVID')
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter(f'{file}_output.avi', fourcc, fps, (width, height))

# 비디오 프레임 처리

count = 1

while True:
    current_frame_pos = int(video.get(cv2.CAP_PROP_POS_FRAMES))

    ret, frame = video.read()

    if not ret:
        time.sleep(3)
        video.release()
        out.release()
        cv2.destroyAllWindows()
        break

    if current_frame_pos % frame_interval == 0 :

        # frame 위에 랙 구역 표시하기 
        color = (0, 255, 0)
        thick, alpha = 2, 0.5
        frame = cv2.rectangle(frame, (u_x1, u_y1), (u_x2, u_y2), color, thick)
        frame = cv2.rectangle(frame, (l_x1, l_y1), (l_x2, l_y2), color, thick)

        # YOLOv8 모델 적용
        results = model(frame)

        # print('================================ results ===================================')
        # print(len(results))
        # print('==================================== 0 ========================================')
        # print(results[0])
        # print('===============================================================================')

        # 잡히는 frame마다 결과 처리 
        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()
            class_ids = result.boxes.cls.cpu().numpy().astype(int)
            speeds = result.speed
            probs = result.probs

            for box, class_id in zip(boxes, class_ids):
                x1, y1, x2, y2 = box
                label = model.names[class_id]
                print(f"좌표: ({x1}, {y1}) - ({x2}, {y2})  라벨: {label}")
                print(f"속도: {speeds}")
                print(f"확률: {probs}")
                
                # 탐지된 객체에 대해 바운딩 박스를 표시하기
                # frame = cv2.rectangle(frame, (x1, y1), (x2, y2), (255,255,0), 1)

                # frame = cv2.rectangle(frame, (l_x1, l_y1), (l_x2, l_y2), color, thick)


                # 뮤팅 알림 표시 
                if label == 'Person' :
                    count += 1
                    print(f"count : {count}")

                    if count > 5 : 
                        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])  # 좌표값을 정수로 변환
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

                        # upper rack
                        if (u_x1 <= x2 <= u_x2) and (u_y1 <= y2 <= u_y2):
                            cv2.putText(frame, 'Person on UPPER RACK', (x1, y2+30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 1)
                            print('person on upper rack')

                        # lower rack  
                        if (l_x1 <= x2 <= l_x2) and (l_y1 <= y2 <= l_y2):
                            cv2.putText(frame, 'Person on LOWER RACK', (x1, y2+30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 1)
                            print('person on lower rack')
                        
                # 작업중 알림 표시
                elif (label == 'Forklift(H)') or (label == 'Forklift(D)') or (label == 'Forklift(V)'): 
                    count = 1          # person용 프레임 카운트 초기화 
                    
                    # upper rack
                    if (u_x1 <= x1 <= u_x2) or (u_x1 <= x2 <= u_x2) or (u_y1 <= y1 <= u_y2) or (u_y1 <= y2 <= u_y2) :
                        cv2.putText(frame, 'Forklift working on UPPER RACK', (x1, y2+30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
                        print('Forklift working on UPPER RACK')

                    # lower rack  
                    if (l_x1 <= x1 <= l_x2) or (l_x1 <= x2 <= l_x2) or (l_y1 <= y1 <= l_y2) or (l_y1 <= y2 <= l_y2) :
                        cv2.putText(frame, 'Forklift working on LOWER RACK', (x1, y2+30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
                        print('Forklift working on LOWER RACK')


                else :
                    count = 1

        # # 감지된 객체에 대한 경계 상자 및 레이블 그리기
        # # annotated_frame = np.squeeze(results.render())

        # # 텍스트 오버레이
        cv2.putText(frame, 'Object Detection With YOLOv8', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

        # # 새로운 비디오 파일에 프레임 저장
        # out.write(frame)

        cv2.imshow('TEST WINDOW', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

video.release()
out.release()
cv2.destroyAllWindows()

Loading runs\detect\train7\weights\best.onnx for ONNX Runtime inference...

0: 640x640 1 Person, 47.6ms
Speed: 0.0ms preprocess, 47.6ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)
좌표: (1077.94873046875, 314.3748779296875) - (1200.576904296875, 504.9385986328125)  라벨: Person
속도: {'preprocess': 0.0, 'inference': 47.594308853149414, 'postprocess': 2.5064945220947266}
확률: None
count : 2


### 모델 불러와서 예측해보기 - 비디오

In [9]:
'''

import cv2
import numpy as np
from ultralytics import YOLO

# 저장된 모델 불러오기
num = 7
model = YOLO(f'./runs/detect/train{num}/weights/best.onnx')

# 비디오 파일 로드
file = "C:/Users/kdp/Desktop/test_video_01.mp4"
video = cv2.VideoCapture(file)

# 새로운 비디오 파일 초기화
fourcc = cv2.VideoWriter_fourcc(*'XVID')
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter(f'{file}_output.avi', fourcc, fps, (width, height))

# 비디오 프레임 처리

count = 1

while True:
    ret, frame = video.read()
    if not ret:
        break

    # YOLOv8 모델 적용
    results = model(frame)

    # 잡히는 frame마다 결과 처리 
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        class_ids = result.boxes.cls.cpu().numpy().astype(int)

        for box, class_id in zip(boxes, class_ids):
            x1, y1, x2, y2 = box
            label = model.names[class_id]
            # print(f"좌표: ({x1}, {y1}) - ({x2}, {y2}) 라벨: {label}")

            if label == 'Person' :
                count += 1

                if count > 5 : 
                    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])  # 좌표값을 정수로 변환
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

                    # upper rack
                    if (u_x1 <= x2 <= u_x2) and (u_y1 <= y2 <= u_y2):
                        cv2.putText(frame, 'Person on UPPER RACK', (x1, y2+30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 1)
                        print('person on upper rack')

                    # lower rack  
                    if (l_x1 <= x2 <= l_x2) and (l_y1 <= y2 <= l_y2):
                        cv2.putText(frame, 'Person on LOWER RACK', (x1, y2+30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 1)
                        print('person on lower rack')
                    
            elif (label == 'Forklift(H)') or (label == 'Forklift(D)') or (label == 'Forklift(V)'): 
                count = 1          # person용 프레임 카운트 초기화 
                
                # upper rack
                if (u_x1 <= x2 <= u_x2) and (u_y1 <= y2 <= u_y2):
                    cv2.putText(frame, 'Forklift working on UPPER RACK', (x1, y2+30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
                    print('Forklift working on UPPER RACK')

                # lower rack  
                if (l_x1 <= x2 <= l_x2) and (l_y1 <= y2 <= l_y2):
                    cv2.putText(frame, 'Forklift working on LOWER RACK', (x1, y2+30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
                    print('Forklift working on LOWER RACK')


            else :
                count = 1

    # 감지된 객체에 대한 경계 상자 및 레이블 그리기
    # annotated_frame = np.squeeze(results.render())

    # 텍스트 오버레이
    cv2.putText(frame, 'Object Detection', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # 새로운 비디오 파일에 프레임 저장
    out.write(frame)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
out.release()
cv2.destroyAllWindows()


'''

'\n\nimport cv2\nimport numpy as np\nfrom ultralytics import YOLO\n\n# 저장된 모델 불러오기\nnum = 7\nmodel = YOLO(f\'./runs/detect/train{num}/weights/best.onnx\')\n\n# 비디오 파일 로드\nfile = "C:/Users/kdp/Desktop/test_video_01.mp4"\nvideo = cv2.VideoCapture(file)\n\n# 새로운 비디오 파일 초기화\nfourcc = cv2.VideoWriter_fourcc(*\'XVID\')\nwidth = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))\nheight = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))\nfps = int(video.get(cv2.CAP_PROP_FPS))\nout = cv2.VideoWriter(f\'{file}_output.avi\', fourcc, fps, (width, height))\n\n# 비디오 프레임 처리\n\ncount = 1\n\nwhile True:\n    ret, frame = video.read()\n    if not ret:\n        break\n\n    # YOLOv8 모델 적용\n    results = model(frame)\n\n    # 잡히는 frame마다 결과 처리 \n    for result in results:\n        boxes = result.boxes.xyxy.cpu().numpy()\n        class_ids = result.boxes.cls.cpu().numpy().astype(int)\n\n        for box, class_id in zip(boxes, class_ids):\n            x1, y1, x2, y2 = box\n            label = model.names[class_id]\n       

In [10]:
'''

# 예측하기 - 사진

file = "F:/video_MNV/6/20231129082547.avi"

results = model.predict(source = file, save=True, conf=0.25)

# source : 예측할 이미지, 비디오 또는 웹캠의 경로나 인덱스를 지정
# save : 예측 결과 저장 여부
# conf : 예측 결과의 신뢰도 임계값을 지정함 

'''

'\n\n# 예측하기 - 사진\n\nfile = "F:/video_MNV/6/20231129082547.avi"\n\nresults = model.predict(source = file, save=True, conf=0.25)\n\n# source : 예측할 이미지, 비디오 또는 웹캠의 경로나 인덱스를 지정\n# save : 예측 결과 저장 여부\n# conf : 예측 결과의 신뢰도 임계값을 지정함 \n\n'

<hr>

In [11]:
# 예측하기 ------------------------------------------------------------------------------

# img_file = "01frame_12696.jpg"
# results = model.predict(source = img_file, save=True, conf=0.25)


# 비디오 예측 ---------------------------------------------------------------------------

# video_file = "20231129115714.avi"
# results = model.predict(source= video_file, save=True, conf=0.25)


# 웹캠 예측  ----------------------------------------------------------------------------

# results = model.predict(source='0', save=True, conf=0.25)  # 0은 기본 웹캠을 의미




# source : 예측할 이미지, 비디오 또는 웹캠의 경로나 인덱스를 지정
# save : 예측 결과 저장 여부
# conf : 예측 결과의 신뢰도 임계값을 지정함 